First reads data
* where are numbers coming from (MACULA ONLY MACULA WITH VESSELS...) ?
* Once you find the optimal shift is it used to match the two immages? where is the trasformation performed? The two are then cropped to make a single couple of matching immages?

In [ ]:
from skimage import io
from skimage.color import rgb2gray
from skimage.transform import resize

import dataset
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DATA = dataset.dataset_path(config_filename='../dataset.txt')

In [ ]:
img_oct = rgb2gray(io.imread(f'{DATA}/{SUBSET}/oct.tif'))
img_maia = rgb2gray(io.imread(f'{DATA}/{SUBSET}/maia.jpg'))

In [ ]:
va_oct = 30
va_maia = 36
oct_px_per_va = img_oct.shape[0] / va_oct
maia_px_per_va = img_maia.shape[0] / va_maia

In [ ]:
print('OCT:', img_oct.shape, 'px/VA:', oct_px_per_va)
print('MAIA:', img_maia.shape, 'px/VA:', maia_px_per_va)

# Finding OCT central template in MAIA

## Template

In [ ]:
MACULA_ONLY = 64
MACULA_WITH_VESSELS = 200
patch_radius = MACULA_ONLY

In [ ]:
def extract_window(image, center, radius):
    assert type(center) == tuple and len(center) == 2, 'Center must be a tuple of window center coordinates'
    if type(radius) == int:
        v_rad = h_rad = radius
    elif type(radius) == tuple and len(radius) == 2:
        v_rad, h_rad = radius
    else:
        raise ValueError('Radius must be an integer or a 2-tuple')
    
    x, y = center
    return image[(x - v_rad):(x + v_rad + 1), (y - h_rad):(y + h_rad + 1)]

In [ ]:
center_coord = img_oct.shape[0] // 2
macula_oct = extract_window(img_oct, (center_coord, center_coord), patch_radius)

In [ ]:
io.imshow(macula_oct)

## NCC matching

In [ ]:
# NCC ritorna una matrice con ogni punto contenente la ncc del template shiftato di x,y
def normalized_cross_correlation(image, template):
    v_template, h_template = template.shape
    assert v_template % 2 == 1 and h_template % 2 == 1, 'Template dimensions must be odd'
    
    demeaned_template = template - np.mean(template)
    var_template = np.sum(np.square(demeaned_template))
    
    v_pad, h_pad = v_template // 2, h_template // 2
    v_image, h_image = image.shape
    
    ncc = np.full(image.shape, fill_value=-1, dtype=np.float32)
    for x in range(v_pad, v_image - v_pad):
        for y in range(h_pad, h_image - h_pad):
            patch = image[x-v_pad:x+v_pad+1, y-h_pad:y+h_pad+1]
            demeaned_patch = patch - np.mean(patch)
            var_patch = np.sum(np.square(demeaned_patch))
            
            ncc[x, y] = np.sum(np.multiply(demeaned_patch, demeaned_template)) / np.sqrt(var_template * var_patch)
    
    return ncc

In [ ]:
crop = (va_maia - va_oct) / 2 * maia_px_per_va
rounded_crop = int(crop)
print(f"Cropping each side by {crop:.2f} (rounded: {rounded_crop})")
height, width = img_maia.shape
img_maia_cropped = img_maia[rounded_crop:(height - rounded_crop), rounded_crop:(width - rounded_crop)]

In [ ]:
io.imsave('../output/cropped.jpg', img_maia_cropped)

In [ ]:
img_maia_resized = resize(img_maia_cropped, img_oct.shape)

In [ ]:
ncc = normalized_cross_correlation(img_maia_cropped, macula_oct)

In [ ]:
io.imshow(ncc)

In [ ]:
#un semplice argmax in 2D
def argmax_image(image):
    index = np.argmax(image.flatten())
    size = image.shape[0]
    x = index // size
    y = index % size
    return x, y

In [ ]:
#una volta trovato il best matching si puo prendere la parte del maia che meglio assomiglia alla oct
x, y = argmax_image(ncc)
macula_maia = img_maia_resized[x-patch_radius:x+patch_radius+1, y-patch_radius:y+patch_radius+1]
print(x, y)

In [ ]:
io.imshow(macula_maia)

In [ ]:
io.imshow(macula_oct)

# Test

In [ ]:
self_ncc = normalized_cross_correlation(img_oct, macula_oct)
self_x, self_y = argmax_image(self_ncc)

In [ ]:
found_macula = extract_window(img_oct, (self_x, self_y), patch_radius)

In [ ]:
print(self_x, self_y)
print('Exact match:', np.array_equal(found_macula, macula_oct))

# Finding MAIA central template in OCT

## Template

In [ ]:
center_coord = img_maia.shape[0] // 2
patch_radius = 80
macula_maia = extract_window(img_maia, (center_coord + 30, center_coord + 5), patch_radius)

In [ ]:
io.imshow(macula_maia)

## NCC matching

# Why this rescaling is used? same what is va ... 

In [ ]:
scaling = oct_px_per_va / maia_px_per_va  # TODO: verify the numbers for VA
target_size = round(macula_maia.shape[0] * scaling)
macula_maia_scaled = resize(macula_maia, (target_size, target_size))

In [ ]:
ncc = normalized_cross_correlation(img_oct, macula_maia_scaled)
center = argmax_image(ncc)
radius = macula_maia_scaled.shape[0] // 2
found_macula_oct = extract_window(img_oct, center, radius)

In [ ]:
io.imshow(ncc)

In [ ]:
io.imshow(found_macula_oct)

In [ ]:
io.imshow(macula_maia)

# Matching based on the optic nerve

In [ ]:
def make_odd(value):
    if value % 2 == 0:
        return value + 1
    else:
        return value
# OPTIC nerve é al centro al lato?
def optic_nerve_patch(image, side='right', vertical_span=0.4, horizontal_span=0.1):
    assert side in ('right', 'left'), "Side must be either 'right' or 'left'"
    
    res = image.shape[0]
    center = res // 2
    
    # Calculate odd absolute span (in pixels)
    abs_h_span, abs_v_span = make_odd(round(horizontal_span * res)), make_odd(round(vertical_span * res))
    abs_v_rad = abs_v_span // 2
    
    if side == 'right':
        return image[center - abs_v_rad:center + abs_v_rad + 1, -abs_h_span:]
    else:
        return image[center - abs_v_rad:center + abs_v_rad + 1, :abs_h_span]

In [ ]:
optic_nerve = optic_nerve_patch(img_oct, side='right')
scaling = maia_px_per_va / oct_px_per_va
h, w = optic_nerve.shape
# TODO: interpolation order
optic_nerve_scaled = resize(optic_nerve, (make_odd(round(scaling * h)), make_odd(round(scaling * w))))

In [ ]:
%%time
ncc = normalized_cross_correlation(img_maia, optic_nerve_scaled)

## Visualize matching accuracy

In [ ]:
optic_nerve_coords = argmax_image(ncc)
h, w = optic_nerve_scaled.shape
maia_optic_nerve = extract_window(img_maia, optic_nerve_coords, (h // 2, w // 2))

minimum = maia_optic_nerve.min()
maia_optic_nerve = (maia_optic_nerve - minimum) / (maia_optic_nerve.max() - minimum)
side_by_side = np.hstack((optic_nerve_scaled, np.flip(maia_optic_nerve, axis=1)))

io.imshow(side_by_side)
plt.title('Optic nerve\nOCT (left) and MAIA (right)')
plt.show()

They seem to be vertically matched correcly, but the patch found in the MAIA image should have more pixels on the optic nerve side area.

## Cropping based on the match

In [ ]:
def crop_with_optic_nerve_patch(image, patch_shape, patch_center, side='right', vertical_span=0.4, horizontal_span=0.1):
    v_patch, h_patch = patch_shape
    v_center_patch, h_center_patch = patch_center
    
    target_side = 784
    target_radius = target_side // 2
    